# 필터링된 검색
- ANN 검색은 지정된 벡터 임베딩과 가장 유사한 벡터 임베딩을 찾음
- 검색 요청에 필터링 조건을 포함하면 Milvus가 ANN 검색을 수행하기 전에 메타데이터 필터링을 수행하여 검색 범위를 전체 컬렉션에서 지정된 필터링 조건과 일치하는 엔티티로만 좁힐 수 있음

참고자료: https://milvus.io/docs/ko/filtered-search.md

## 개요
- Milvus에서 필터링된 검색은 필터링이 적용되는 단계에 따라 표준 필터링과 반복 필터링의 두 가지 유형으로 분류됨

### 표준 필터링
- 컬렉션에 벡터 임베딩과 해당 메타데이터가 모두 포함된 경우, ANN 검색 전에 메타데이터를 필터링하여 검색 결과의 관련성을 향상시킬 수 있음
- Milvus는 필터링 조건이 포함된 검색 요청을 받으면 지정된 필터링 조건과 일치하는 엔티티 내에서 검색 범위를 제한함

### 반복 필터링
- 표준 필터링 프로세스는 검색 범위를 작은 범위로 효과적으로 좁혀줌
- 그러나 필터링 표현식이 지나치게 복잡하면 검색 지연 시간이 매우 길어짐
- 이러한 경우 반복 필터링이 대안이 될 수 있으며, 스칼라 필터링의 작업 부하를 줄이는 데 도움이 됨

In [28]:
from pymilvus import MilvusClient, DataType

# 클라이언트 연결
client = MilvusClient(uri="http://localhost:19530")

if client.has_collection("my_collection"):
    client.drop_collection("my_collection")

# 정확하게 정의된 스키마
schema = client.create_schema(
    auto_id=False,
    enable_dynamic_schema=True
)

schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True, auto_id=False)
schema.add_field(field_name="vector", datatype=DataType.FLOAT_VECTOR, dim=5)
schema.add_field(field_name="color", datatype=DataType.VARCHAR, max_length=32)
schema.add_field(field_name="likes", datatype=DataType.INT64)

index_params = client.prepare_index_params()
index_params.add_index(field_name="vector", index_type="AUTOINDEX", metric_type="IP")

# 컬렉션 생성
client.create_collection(
    collection_name="my_collection",
    vector_field_name="vector",
    schema=schema,
    index_params=index_params
)

In [29]:
data = [
    {"id": 0, "vector": [0.3580376395471989, -0.6023495712049978, 0.18414012509913835, -0.26286205330961354, 0.9029438446296592], "color": "pink_8682", "likes": 165},
    {"id": 1, "vector": [0.19886812562848388, 0.06023560599112088, 0.6976963061752597, 0.2614474506242501, 0.838729485096104], "color": "red_7025", "likes": 25},
    {"id": 2, "vector": [0.43742130801983836, -0.5597502546264526, 0.6457887650909682, 0.7894058910881185, 0.20785793220625592], "color": "orange_6781", "likes": 764},
    {"id": 3, "vector": [0.3172005263489739, 0.9719044792798428, -0.36981146090600725, -0.4860894583077995, 0.95791889146345], "color": "pink_9298", "likes": 234},
    {"id": 4, "vector": [0.4452349528804562, -0.8757026943054742, 0.8220779437047674, 0.46406290649483184, 0.30337481143159106], "color": "red_4794", "likes": 122},
    {"id": 5, "vector": [0.985825131989184, -0.8144651566660419, 0.6299267002202009, 0.1206906911183383, -0.1446277761879955], "color": "yellow_4222", "likes": 12},
    {"id": 6, "vector": [0.8371977790571115, -0.015764369584852833, -0.31062937026679327, -0.562666951622192, -0.8984947637863987], "color": "red_9392", "likes": 58},
    {"id": 7, "vector": [-0.33445148015177995, -0.2567135004164067, 0.8987539745369246, 0.9402995886420709, 0.5378064918413052], "color": "grey_8510", "likes": 775},
    {"id": 8, "vector": [0.39524717779832685, 0.4000257286739164, -0.5890507376891594, -0.8650502298996872, -0.6140360785406336], "color": "white_9381", "likes": 876},
    {"id": 9, "vector": [0.5718280481994695, 0.24070317428066512, -0.3737913482606834, -0.06726932177492717, -0.6980531615588608], "color": "purple_4976", "likes": 765}
]

client.insert(
    collection_name="my_collection",
    data=data
)

{'insert_count': 10, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'cost': 0}

In [30]:
query_vector = [0.3580376395471989, -0.6023495712049978, 0.18414012509913835, -0.26286205330961354, 0.9029438446296592]


res = client.search(
    collection_name="my_collection",
    data=[query_vector],
    limit=5,
    # highlight-start
    filter='color like "red%" and likes > 50',
    output_fields=["color", "likes"]
    # highlight-end
)

for hits in res:
    print("TopK results:")
    for hit in hits:
        print(hit)


TopK results:
{'id': 4, 'distance': 0.9902133941650391, 'entity': {'color': 'red_4794', 'likes': 122}}
{'id': 6, 'distance': -0.4113418757915497, 'entity': {'color': 'red_9392', 'likes': 58}}


## 반복 필터링으로 검색

In [31]:
from pymilvus import MilvusClient

query_vector = [0.3580376395471989, -0.6023495712049978, 0.18414012509913835, -0.26286205330961354, 0.9029438446296592]

res = client.search(
    collection_name="my_collection",
    data=[query_vector],
    limit=5,
    # highlight-start
    filter='color like "red%" and likes > 50',
    output_fields=["color", "likes"],
    search_params={
        "hints": "iterative_filter"
    }
    # highlight-end
)

for hits in res:
    print("TopK results:")
    for hit in hits:
        print(hit)


TopK results:
{'id': 4, 'distance': 0.9902133941650391, 'entity': {'color': 'red_4794', 'likes': 122}}
{'id': 6, 'distance': -0.4113418757915497, 'entity': {'color': 'red_9392', 'likes': 58}}
